In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import hopsworks
import pandas as pd
from datetime import timedelta
from src.inference import get_feature_store, fetch_predictions

def fetch_hourly_rides(hours):
    current_hour = (pd.Timestamp.now(tz="Etc/UTC") - timedelta(hours=hours)).floor('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_NAME,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour >= current_hour)

    return query.read()

c:\Users\vidyu\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df = fetch_hourly_rides(12)

2025-03-03 13:45:35,457 INFO: Initializing external client
2025-03-03 13:45:35,457 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 13:45:36,982 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215639
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (57.32s) 


In [5]:
df

,pickup_hour,pickup_location_id,rides
0,2025-03-03 01:00:00+00:00,71,0
1,2025-03-03 01:00:00+00:00,77,0
2,2025-03-03 01:00:00+00:00,62,0
3,2025-03-03 01:00:00+00:00,2,0
4,2025-03-03 01:00:00+00:00,80,0
...,...,...,...
497,2025-03-03 02:00:00+00:00,6,0
498,2025-03-03 02:00:00+00:00,30,0
499,2025-03-03 02:00:00+00:00,184,0
500,2025-03-03 02:00:00+00:00,62,0


In [6]:
df_pred = fetch_predictions(12)

2025-03-03 13:46:42,016 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 13:46:42,021 INFO: Initializing external client
2025-03-03 13:46:42,021 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 13:46:43,110 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215639
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (55.10s) 


In [7]:
df_pred

,pickup_location_id,predicted_demand,pickup_hour
0,33,1.0,2025-03-03 02:00:00+00:00
1,62,1.0,2025-03-03 02:00:00+00:00
2,34,0.0,2025-03-03 02:00:00+00:00
3,61,2.0,2025-03-03 02:00:00+00:00
4,220,0.0,2025-03-03 02:00:00+00:00
...,...,...,...
748,183,0.0,2025-03-03 19:00:00+00:00
749,82,1.0,2025-03-03 19:00:00+00:00
750,135,0.0,2025-03-03 19:00:00+00:00
751,34,0.0,2025-03-03 19:00:00+00:00


In [8]:
merged_df = pd.merge(df, df_pred, on=['pickup_location_id', 'pickup_hour'])

In [9]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand
0,2025-03-03 02:00:00+00:00,230,17,124.0
1,2025-03-03 02:00:00+00:00,227,0,0.0
2,2025-03-03 02:00:00+00:00,221,0,0.0
3,2025-03-03 02:00:00+00:00,236,0,257.0
4,2025-03-03 02:00:00+00:00,7,0,3.0
...,...,...,...,...
246,2025-03-03 02:00:00+00:00,6,0,-0.0
247,2025-03-03 02:00:00+00:00,30,0,0.0
248,2025-03-03 02:00:00+00:00,184,0,0.0
249,2025-03-03 02:00:00+00:00,62,0,1.0


In [10]:
merged_df['difference'] = merged_df['predicted_demand'] - merged_df['rides']

In [11]:
merged_df.sort_values(["pickup_location_id", "pickup_hour"])

,pickup_hour,pickup_location_id,rides,predicted_demand,difference
36,2025-03-03 02:00:00+00:00,2,0,0.0,0.0
123,2025-03-03 02:00:00+00:00,3,0,0.0,0.0
129,2025-03-03 02:00:00+00:00,4,1,5.0,4.0
246,2025-03-03 02:00:00+00:00,6,0,-0.0,-0.0
4,2025-03-03 02:00:00+00:00,7,0,3.0,3.0
...,...,...,...,...,...
22,2025-03-03 02:00:00+00:00,259,0,0.0,0.0
199,2025-03-03 02:00:00+00:00,260,0,1.0,1.0
243,2025-03-03 02:00:00+00:00,261,0,23.0,23.0
37,2025-03-03 02:00:00+00:00,262,1,23.0,22.0


In [12]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand,difference
0,2025-03-03 02:00:00+00:00,230,17,124.0,107.0
1,2025-03-03 02:00:00+00:00,227,0,0.0,0.0
2,2025-03-03 02:00:00+00:00,221,0,0.0,0.0
3,2025-03-03 02:00:00+00:00,236,0,257.0,257.0
4,2025-03-03 02:00:00+00:00,7,0,3.0,3.0
...,...,...,...,...,...
246,2025-03-03 02:00:00+00:00,6,0,-0.0,-0.0
247,2025-03-03 02:00:00+00:00,30,0,0.0,0.0
248,2025-03-03 02:00:00+00:00,184,0,0.0,0.0
249,2025-03-03 02:00:00+00:00,62,0,1.0,1.0


In [13]:
import pandas as pd  
import plotly.express as px
df1 = df
df2 = df_pred

# Merge the DataFrames on 'pickup_location_id' and 'pickup_hour'  
merged_df = pd.merge(df1, df2, on=['pickup_location_id', 'pickup_hour'])  

# Calculate the absolute error  
merged_df['absolute_error'] = abs(merged_df['predicted_demand'] - merged_df['rides'])  

# Group by 'pickup_hour' and calculate the mean absolute error (MAE)  
mae_by_hour = merged_df.groupby('pickup_hour')['absolute_error'].mean().reset_index()  
mae_by_hour.rename(columns={'absolute_error': 'MAE'}, inplace=True)  

# Create a Plotly plot  
fig = px.line(  
    mae_by_hour,  
    x='pickup_hour',  
    y='MAE',  
    title='Mean Absolute Error (MAE) by Pickup Hour',  
    labels={'pickup_hour': 'Pickup Hour', 'MAE': 'Mean Absolute Error'},  
    markers=True  
)  

# Show the plot  
fig.show()

In [14]:
mae_by_hour["MAE"].mean()

13.171314741035857